## Autotune

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence

# Helper libraries
import numpy as np
import time

from keras_tuner.tuners import RandomSearch,Hyperband,BayesianOptimization
from keras_tuner.engine.hypermodel import HyperModel
from keras_tuner.engine.hyperparameters import HyperParameters

#tf.compat.v1.disable_eager_execution() // to disable eager
print(tf.__version__)
print(tf.config.experimental.list_physical_devices('GPU'))



2.6.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### define model & learning hyperparameters

In [2]:
data_dir = "data/Augmented"

batch_size = 32
img_height = 100
img_width = 100

epochs = 100

#BUFFER_SIZE = BATCH_SIZE*20
#EVALUATION_INTERVAL = round(len(labels)/BATCH_SIZE)
#EVALUATION_TIME = round(EVALUATION_INTERVAL / 10)

## Load and Preprocess Dataset

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1365 files belonging to 8 classes.
Using 1092 files for training.
Found 1365 files belonging to 8 classes.
Using 273 files for validation.


In [4]:
class_names = train_ds.class_names
print(class_names)

num_classes = len(class_names)

#AUTOTUNE
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

['1', '2', '3', '4', '5', '6', '7', 'Overripen']


## Build the model

In [7]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)))
    for i in range(hp.Int('num_Conv_layers', 3, 4)):
    #for i in range(4):
        model.add(keras.layers.Conv2D(hp.Int('units_Conv_' + str(i),
                                            min_value=32,
                                            max_value=256,
                                            default=32,
                                            step=32),
                               3, padding='same',activation='relu'))
        model.add(keras.layers.MaxPooling2D())
        model.add(keras.layers.Dropout(rate=hp.Float('rate_DO_Conv_' + str(i),
                                min_value=0.0,
                                max_value=0.4,
                                default=0.0,
                                step=0.1)))
    model.add(keras.layers.Dense(units=hp.Int('units_Dense',
                                        min_value=16,
                                        max_value=256,
                                        default=16,
                                        step=16),
                           activation='relu'))
    model.add(keras.layers.Dropout(rate=hp.Float('rate_DO_dense',
                                        min_value=0.0,
                                        max_value=0.2,
                                        default=0.0,
                                        step=0.05)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(num_classes))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[5e-6,1e-6, 5e-5, 1e-5])),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=1000,
    executions_per_trial=1,
    directory='tuner',
    project_name='run1')

tuner.search_space_summary()

INFO:tensorflow:Reloading Oracle from existing project tuner\run1\oracle.json
INFO:tensorflow:Reloading Tuner from tuner\run1\tuner0.json
Search space summary
Default search space size: 10
num_Conv_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
units_Conv_0 (Int)
{'default': 32, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
rate_DO_Conv_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.4, 'step': 0.1, 'sampling': None}
units_Dense (Int)
{'default': 16, 'conditions': [], 'min_value': 16, 'max_value': 256, 'step': 16, 'sampling': None}
rate_DO_dense (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.2, 'step': 0.05, 'sampling': None}
learning_rate (Choice)
{'default': 0.0005, 'conditions': [], 'values': [0.0005, 0.0001, 5e-05, 1e-05], 'ordered': True}
units_Conv_1 (Int)
{'default': 32, 'conditions': [], 'min_value': 32, 'max_value': 256,

## Train the model

In [8]:
tuner.search(train_ds, epochs=epochs,
      validation_data=val_ds)
tuner.results_summary()


INFO:tensorflow:Oracle triggered exit
Results summary
Results in tuner\run1
Showing 10 best trials
Trial summary
Hyperparameters:
num_Conv_layers: 3
units_Conv_0: 192
rate_DO_Conv_0: 0.0
units_Dense: 160
rate_DO_dense: 0.05
learning_rate: 5e-05
units_Conv_1: 256
rate_DO_Conv_1: 0.2
units_Conv_2: 32
rate_DO_Conv_2: 0.2
Score: 0.8974359035491943
Trial summary
Hyperparameters:
num_Conv_layers: 3
units_Conv_0: 64
rate_DO_Conv_0: 0.0
units_Dense: 144
rate_DO_dense: 0.15000000000000002
learning_rate: 0.0005
units_Conv_1: 128
rate_DO_Conv_1: 0.30000000000000004
units_Conv_2: 128
rate_DO_Conv_2: 0.2
Score: 0.8937729001045227
Trial summary
Hyperparameters:
num_Conv_layers: 3
units_Conv_0: 224
rate_DO_Conv_0: 0.1
units_Dense: 64
rate_DO_dense: 0.15000000000000002
learning_rate: 0.0001
units_Conv_1: 160
rate_DO_Conv_1: 0.30000000000000004
units_Conv_2: 96
rate_DO_Conv_2: 0.4
Score: 0.8901098966598511
Trial summary
Hyperparameters:
num_Conv_layers: 3
units_Conv_0: 160
rate_DO_Conv_0: 0.0
units_Den